In [1]:
import preprocessor as p
import numpy as np
from textblob import TextBlob
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
import pandas as pd
import numpy as np
import gensim
import re
from tqdm import tqdm
from bs4 import BeautifulSoup


In [2]:
tqdm.pandas()

Read CSV

Drop useless columns

In [3]:
df =pd.read_csv("tweets/jan.csv")
df.head()
data = df.drop(["mentions","to","geo","permalink","hashtags"],axis=1)

Clean function using a combination of re and preprocessor

In [4]:
def cleaning(text):
    cleantext = p.clean(text)
    cleantext = re.sub('[^a-zA-Z ]','',cleantext)
    return cleantext
def clean(data):
    #cleanr = re.compile("<.*?>")
    cleanr= re.sub(r"http\S+", "", data)
    
    cleanr= re.sub(r"pic.\S+", "", cleanr)
    #cleanr= re.sub(cleanr, "", data)
    #cleantext = re.sub(cleanr,'',data)
    cleantext = re.sub('[^a-zA-Z# ]','',cleanr)
    clean_hash = re.sub(r"#\S+","",cleantext)
    
    return clean_hash
data["text"] = df.progress_apply(lambda x:clean(x["text"]),axis=1)

data.head()

100%|██████████| 579512/579512 [00:11<00:00, 51358.92it/s]


,date,username,replies,retweets,favorites,text,id
0,2019-01-10 23:59:58,CryptoPressNews,0,0,1,BTC bitcoins Current Price h h ...,1083513813746442240
1,2019-01-10 23:59:57,sajmathieu,1,0,1,Funny that you say BIP had broad support yet m...,1083513809136951297
2,2019-01-10 23:59:57,Libreinfo,0,0,0,Price Watch Currency Sinks as New Year Hope S...,1083513807383584769
3,2019-01-10 23:59:54,CoinSpotterEU,0,0,0,Crypto Visionary Nick Szabo Says Banks May Scr...,1083513795027308544
4,2019-01-10 23:59:53,profits_bitcoin,0,1,1,BTC Bitcoin Magazine Research Firm Charts Unsp...,1083513790648512513


tokenize and stemm and then join text

In [5]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import LancasterStemmer
from nltk.corpus import stopwords
porter = PorterStemmer()
lancaster=LancasterStemmer()
def stemSentence(sentence):
    token_words=word_tokenize(sentence)
    token_words
    stem_sentence=[]
    filtered_sentence = [w for w in token_words if not w in stopwords.words()]
            
    for word in filtered_sentence:
        stem_sentence.append(porter.stem(word))
        stem_sentence.append(" ")
    return "".join(stem_sentence)

x=stemSentence(data["text"][5])
print(x)
print(data["text"][5])

swiss stock exchang bring crypto center global financi industri 
Swiss Stock Exchange Bringing Crypto to the Center of the Global Financial Industry        


TextBlob functions

In [6]:
data["subjectivity"] = np.zeros((data.shape[0],1))
data["blob_sent"] = np.zeros((data.shape[0],1))
def textblob_sent(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[0]
def textblob_sub(text):
    text = stemSentence(text)
    analysis = TextBlob(str(text))
    return analysis.sentiment[1]
    


In [7]:
print(textblob_sent(df["text"][5]))
print(textblob_sent(stemSentence(df["text"][5])))

-0.05
-0.05


In [8]:
data["blob_sent"] = df.progress_apply(lambda x:textblob_sent(x["text"]),axis=1)
data["subjectivity"] = df.progress_apply(lambda x:textblob_sub(x["text"]),axis=1)


100%|██████████| 579512/579512 [12:12:03<00:00, 13.19it/s]  


Vader sentiment

In [9]:
sid = SentimentIntensityAnalyzer()
def vader(text):
    score = sid.polarity_scores(text)
    return score["compound"]


In [10]:
data["vader_sent"] = df.progress_apply(lambda x:vader(x["text"]),axis=1)
data.head(10)

100%|██████████| 579512/579512 [02:18<00:00, 4197.86it/s]


,date,username,replies,retweets,favorites,text,id,subjectivity,blob_sent,vader_sent
0,2019-01-10 23:59:58,CryptoPressNews,0,0,1,BTC bitcoins Current Price h h ...,1083513813746442240,0.450000,0.250000,0.0000
1,2019-01-10 23:59:57,sajmathieu,1,0,1,Funny that you say BIP had broad support yet m...,1083513809136951297,0.315278,-0.090278,0.7490
2,2019-01-10 23:59:57,Libreinfo,0,0,0,Price Watch Currency Sinks as New Year Hope S...,1083513807383584769,0.454545,0.136364,0.4404
3,2019-01-10 23:59:54,CoinSpotterEU,0,0,0,Crypto Visionary Nick Szabo Says Banks May Scr...,1083513795027308544,0.000000,0.000000,0.5267
4,2019-01-10 23:59:53,profits_bitcoin,0,1,1,BTC Bitcoin Magazine Research Firm Charts Unsp...,1083513790648512513,0.400000,-0.200000,0.0000
5,2019-01-10 23:59:51,CoinSpotterEU,0,0,0,Swiss Stock Exchange Bringing Crypto to the Ce...,1083513785103585280,0.050000,-0.050000,0.0000
6,2019-01-10 23:59:50,CoinSpotterEU,0,0,0,January Cryptocurrency Predictions Report Bitc...,1083513780019945472,0.250000,0.250000,0.0000
7,2019-01-10 23:59:44,jpthor__,9,10,46,Synced a new bitcoin node on gbgb SSDCore VM S...,1083513754627657728,0.612626,-0.171212,-0.4939
8,2019-01-10 23:59:40,CoinSpotterEU,0,0,0,Bitwise Files Bitcoin ETF With SEC Another ETF...,1083513737426976768,0.600000,-0.100000,0.5106
9,2019-01-10 23:59:36,bitlyfool,0,0,0,Bitcoin Plunges Amidst Overall Crypto Market D...,1083513720922267648,0.000000,0.000000,0.0000


In [11]:
adwords = ["give","referral","referal","bonus","signup","invite","freebitcoin","lucky",
          "lottery","giveaway","coupon","prizes","enter","airdrop","game","contest","bonuses"]
def contains_word(s, w):
    return (' ' + w + ' ') in (' ' + s + ' ')
def adchecker(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return count
def adword(text):
    words = []
    count = 0
    for i in adwords:
        if (contains_word(text,i)):
            count+= 1
            words.append(i)
    return words

In [12]:
data["ad_count"] = np.zeros((data.shape[0],1))
data["adwords"] = np.zeros((data.shape[0],1))

data["ad_count"] = df.progress_apply(lambda x:adchecker(x["text"]),axis=1)
data["adwords"] = df.progress_apply(lambda x:adword(x["text"]),axis=1)

data.to_csv("clean100.csv")


100%|██████████| 579512/579512 [00:11<00:00, 52642.97it/s]
